In [ ]:
import matplotlib.pyplot as plt
import cantera as ct
import pandas as pd

In [ ]:
# ------------------------------------------------------------
# Input
# ------------------------------------------------------------
path_data = "path/to/data/data_SH2_vs_Z_....csv"
path_mech = "path/to/mech_H2Burke.yaml"

# Reference 
T = 298
p = 1 * ct.one_atm

# High temperature
#T = 500
#p = 1 * ct.one_atm

# High pressure
#T = 298
#p = 5 * ct.one_atm

# ------------------------------------------------------------

columns = ["TimeStep","Time","SH2  ", "TEMPE","Z","x","y","z"]

In [ ]:
data = pd.read_csv(path_data, names=columns, skiprows=1)
print(len(data))
data.head()

In [ ]:


X_fuel = {"H2": 1.0}
X_ox = {"O2": 1.0, "N2": 3.76}
gas = ct.Solution(path_mech)

gas.TP = T, p
gas.set_equivalence_ratio(phi=0.5, fuel=X_fuel, oxidizer=X_ox)

flame = ct.FreeFlame(gas, width=0.3)
flame.transport_model = "mixture-averaged"

flame.energy_enabled = True
flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

flame.set_max_jac_age(10, 10)
flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
flame.max_time_step_count = 5000    

# Set steady-state tolerance to relative and absolute tolerances of 1e-15
flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])
flame.solve(loglevel=0, auto=True)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.scatter(data["TEMPE"], data["SH2  "], label="Data")
ax.plot(
    flame.T, 
    gas.molecular_weights[gas.species_index("H2")] * flame.net_production_rates[gas.species_index("H2"), :], 
    label="1D flame", color="black")
ax.set_xlabel(r"$T$ $[K]$")
ax.set_ylabel(r"$\dot{\omega}_{H_2}$ $[kg/m^3 s]$")
ax.legend()